# Laboratorio 2

In [1]:
import numpy as np
import random


1. Implementar una función en Python que genere un arreglo de números aleatorios entre 0  y 9 (1.0 puntos)

Requerimientos:
*   Entradas: tamaño del arreglo
*   Salida: arreglo de numpy con elementos de 32 bits

In [ ]:
def generarArreglo(N):
    #Que sea de 32 bits
    arreglo = np.random.randint(0,10,N,dtype=np.int32)
    return arreglo

print(generarArreglo(20))


2.   Implementar una función en Python que detecte los números pares de un arreglo (1.0 puntos)

Requerimientos:



*   Entrada: arreglo de numpy con elementos de 32 bits
*   Salida: arreglo de numpy con elementos de 8 bits
*   Cuando el número es par el valor del arreglo de salida en la posición de ese elemento será 0, y cuando es impar será 1.




In [ ]:
import ctypes
import numpy as np
import random

arregloPrueba = generarArreglo(20)
longitudArreglo = len(arregloPrueba)

def parImparArreglo(arreglo, longitudArreglo):
    #Si el numero es par, sera 0 en esa posicion y si es impar sera 1
    for i in range(longitudArreglo):
        if arreglo[i]%2 == 0:
            arreglo[i] = 0
        else:
            arreglo[i] = 1
    
    #Regresar el arreglo como arreglo de 8 bits
    return arreglo.astype(np.int8)
    

print(arregloPrueba)
print(parImparArreglo(arregloPrueba, longitudArreglo))



3.   Implementar una función en C que detecte los números pares de un arreglo (2.0 puntos)

Requerimientos:



*   Entradas: 
    + Puntero a los elementos de entrada enteros de 32 bits
    + Cantidad de elementos
    + Puntero a los elementos de salida enteros de 8 bits





In [ ]:
%%file parImparArregloC.c
//Funcion en la que ingresa un arreglo de 32 bits y retorna un arreglo 
//En el que se pondra 0 si el numero es par, 1 si es impar
//Entradas un arreglo entero de 32 bits, la longitud y un arreglo entero de 8 bits
void parImparArregloC(int *arreglo,int longitud, char *arregloNuevo){
    for(int i=0;i<longitud;i++){
        if(arreglo[i]%2 == 0){
            arregloNuevo[i] = 0;
        }else{
            arregloNuevo[i] = 1;
        }
    }
}



4.   Generar el *object file* y la *shared library* (1.0 puntos)



In [ ]:
! gcc -c -Wall -Werror -fpic parImparArregloC.c
#para crear la shared library
! gcc -shared parImparArregloC.o -o libParImparArreglo.so



5.   Hacer una función que vincule la librería dinámica de C con Python (1.0 puntos)



In [ ]:
def vincularLibreriaC():
    lib = ctypes.CDLL('./libParImparArreglo.so')
    lib.parImparArregloC.argtypes = [np.ctypeslib.ndpointer(dtype=np.int32), ctypes.c_int, np.ctypeslib.ndpointer(dtype=np.int8)]
    return lib



6.   Envolver la función implementada en C para que cumpla con los mismos requerimientos que la función implementada en Pyhon (1.0 puntos)



In [ ]:
def detectarParC(arreglo):
    N = len(arreglo)   
    arregloNuevo = np.zeros(N).astype(np.int8)
    lib = vincularLibreriaC()
    lib.parImparArregloC(arreglo, N, arregloNuevo)
    return arregloNuevo


7.   Implementar una función en ASM que detecte los números pares de un arreglo (4.0 puntos)

Requerimientos:



*   Entradas: 
    + Puntero a los elementos de entrada enteros de 32 bits
    + Cantidad de elementos
    + Puntero a los elementos de salida enteros de 8 bits

In [ ]:
%%file parImparArregloASM.asm
    global parImparArregloASM
    section .text
parImparArregloASM:
;rdi = arreglo
;rsi = longitud
;rdx = arregloNuevo
    mov rax, rsi ; rax = longitud
    mov rcx, 0  ; rcx = i
    mov r8, 0   ; r8 = 0
    for:
        cmp rcx, rax ; i == longitud?
        je salida
        mov r9, [rdi + rcx*4] ; r9 = arreglo[i]
        mov r10, 2 ; r10 = 2
        mov r11, 0 ; r11 = 0
        div r10 ; r9 = r9/2
        cmp rdx, r11 ; rdx == 0?
        je par
        jmp impar
    par:
        mov [rdx + rcx*2], r8 ; arregloNuevo[i] = 0
        jmp siguiente
    impar:
        mov r8, 1 ; r8 = 1
        mov [rdx + rcx*2], r8 ; arregloNuevo[i] = 1
    siguiente:
        inc rcx ; i++
        jmp for
        


salida:
    ret
    






8.   Generar el *object file* y la *shared library* (1.0 puntos)

In [ ]:
!nasm -f elf64 parImparArregloASM.asm -o parImparArregloASM.o
!gcc -shared parImparArregloC.c parImparArregloASM.o -o parImparArreglo.so

9.   Hacer una función que vincule la librería dinámica de ASM con Python (1.0 puntos)

In [ ]:
def vincularASM():
    lib = ctypes.CDLL('./parImparArreglo.so')
    lib.parImparArregloASM.argtypes = [np.ctypeslib.ndpointer(dtype=np.int32), ctypes.c_int, np.ctypeslib.ndpointer(dtype=np.int8)]
    return lib

10.   Envolver la función implementada en ASM para que cumpla con los mismos requerimientos que la función implementada en Pyhon (1.0 puntos)


In [ ]:
def detectarParASM(arreglo):
    N = len(arreglo)   
    arregloNuevo = np.zeros(N).astype(np.int8)
    lib = vincularASM()
    lib.parImparArregloASM(arreglo, N, arregloNuevo)
    return arregloNuevo



11.   Validar que las tres funciones dan el mismo resultado para un tamaño de `16384` elementos. (1.0 puntos)



In [ ]:

arregloPrueba = generarArreglo(1024)
arregloPython = parImparArreglo(arregloPrueba, len(arregloPrueba))
arregloC = detectarParC(arregloPrueba)
arregloASM = detectarParASM(arregloPrueba)

print("Python: ", arregloPython)
print("C: ", arregloC)
print("ASM: ", arregloASM)





12.   Realice mediciones de tiempo para las funciones implementadas. Las mediciones serán  para la lista de tamaños `[32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384]`. Para cada tamaño realice 100 mediciones y filtre sus mediciones con la función `filtro_mediana` con una ventana de 7. Emplee el método `perf_counter` del módulo `time`. (1.0 puntos)



In [ ]:
import statistics
import time 

def filtro_mediana(sign, ventana):
    signc = []
    tam = len(sign)
    offs = int((ventana-1)/2)
    for i in range(tam):
        inicio = i - offs if (i - offs > 0) else i
        fin = i + offs if (i + offs < tam) else tam
        signc.append(statistics.median(sign[inicio:fin]))
    return signc

L = [32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384]
listaPyFinal = []
listaCFinal = []
listaASMFinal = []

for N,i in L:
    arregloPrueba = np.random.randint(1,N).astype(np.int32)
    listaPy = []
    listaC = []
    listaASM = []
    it = 100
    for i in range(it):
        tic =time.perf_counter()
        arregloPython = parImparArreglo(arregloPrueba, len(arregloPrueba))
        toc = time.perf_counter()
        listaPy.append((toc-tic)*1e6)

        tic = time.perf_counter()
        arregloC = detectarParC(arregloPrueba)
        toc = time.perf_counter()
        listaC.append((toc-tic)*1e6)

        tic = time.perf_counter()
        arregloASM = detectarParASM(arregloPrueba)
        toc = time.perf_counter()
        listaASM.append((toc-tic)*1e6)
    listaPyFinal[i] = statistics.mean(filtro_mediana(listaPy, 7))
    listaCFinal[i] = statistics.mean(filtro_mediana(listaC, 7))
    listaASMFinal[i] = statistics.mean(filtro_mediana(listaASM, 7))
    
print("Python: ", statistics.mean(listaPyFinal))
print("C: ", statistics.mean(listaCFinal))
print("ASM: ", statistics.mean(listaASMFinal))






13.   Presente las siguientes gráficas (4.0 puntos):
     +   Tiempo vs Tamaño
        - Una gráfica en la que estén Python, C y ASM
        - Una gráfica en la que estén solo C y ASM
     +   Speedup vs Tamaño
        - Una gráfica del speedup de C y ASM respecto de Python
        - Una gráfica del speedup de ASM respecto de C




In [ ]:
import matplotlib.pyplot as plt

#Para mostrar en Python, C y ASM
plt.plot(L, listaPyFinal, label = "Python", color = "red")
plt.plot(L, listaCFinal, label = "C", color = "green")
plt.plot(L, listaASMFinal, label = "ASM", color = "blue")
plt.savefig("graficaPy_C_ASM.png")

#Para mostrar en ASM y C
plt.plot(L, listaCFinal, label = "C", color = "green")
plt.plot(L, listaASMFinal, label = "ASM", color = "blue")
plt.savefig("graficaC_ASM.png")